In [3]:
# Load necessary libraries
library(ggplot2)
library(dplyr)
library(caTools)
library(car)
library(carat)
library(shiny)
library(tidyverse)
library(randomForest)
library(lattice)
library(caret)
library(shinyjs)
library(shinythemes)  # Modern theme support
library(plotly)  # Interactive plots

## Back-end Logic

In [10]:
# Load dataset
df <- read.csv("C:/Users/DJ COMPUTERS/OneDrive/Desktop/PricePredX/AmesHousing.csv")

# Data Preprocessing
df <- df %>%
  select(SalePrice, Lot.Area, Overall.Qual, Year.Built, Gr.Liv.Area, Full.Bath, Garage.Cars, Kitchen.Qual) %>%
  mutate(Kitchen.Qual = factor(Kitchen.Qual, levels = c("Fa", "TA", "Gd", "Ex"), ordered = TRUE))

df <- na.omit(df)

# Train-Test Split
set.seed(42)
trainIndex <- createDataPartition(df$SalePrice, p = 0.8, list = FALSE)
trainData <- df[trainIndex, ]
testData <- df[-trainIndex, ]

# Train Model (Random Forest)
model <- randomForest(SalePrice ~ ., data = trainData, ntree = 150, importance = TRUE)

# Define ROI Impact Data
renovation_costs <- data.frame(
  Renovation = c("Kitchen Upgrade", "Garage Addition", "Basement Finish", "Better Lot Area Utilization"),
  Cost = c(50000, 30000, 20000, 10000),  # estimated costs in dollars
  Impact = c(0.10, 0.07, 0.05, 0.03)  # estimated price increase as a percentage
)

## Shiny App Implementation

In [ ]:
# SHINY UI - Modernized
ui <- fluidPage(
  useShinyjs(),
  theme = shinytheme("darkly"),  # Modern UI theme
  
  titlePanel(tags$h2("🏡 AI-Powered Home Valuation & Renovation Advisor", align = "center")),
  
  sidebarLayout(
    sidebarPanel(
      h4("🏠 Property Details"),
      numericInput("lot_area", "Lot Area (sqft):", 8000),
      sliderInput("overall_qual", "Overall Quality:", min = 1, max = 10, value = 5),
      numericInput("year_built", "Year Built:", 2000),
      numericInput("gr_liv_area", "Living Area (sqft):", 1500),
      numericInput("full_bath", "Full Bathrooms:", 2),
      numericInput("garage_cars", "Garage Capacity:", 2),
      selectInput("kitchen_qual", "Kitchen Quality:", choices = c("Fair" = "Fa", "Typical" = "TA", "Good" = "Gd", "Excellent" = "Ex")),
      actionButton("predict", "🔮 Predict Price", class = "btn-primary")
    ),
    
    mainPanel(
      tags$h3("🏡 Property Valuation"),
      textOutput("predicted_price"),
      textOutput("renovation_suggestion"),
      textOutput("roi_estimate"),
      
      hr(),
      tags$h3("📊 ROI Analysis - Renovation Impact"),
      plotlyOutput("roi_plot")  # Retained this section
    )
  )
)

# SHINY SERVER - Improved Logic
server <- function(input, output, session) {
  
  observeEvent(input$predict, {
    input_data <- data.frame(
      Lot.Area = input$lot_area,
      Overall.Qual = input$overall_qual,
      Year.Built = input$year_built,
      Gr.Liv.Area = input$gr_liv_area,
      Full.Bath = input$full_bath,
      Garage.Cars = input$garage_cars,
      Kitchen.Qual = factor(input$kitchen_qual, levels = c("Fa", "TA", "Gd", "Ex"), ordered = TRUE)
    )
    
    predicted_price <- predict(model, newdata = input_data)
    
    # Renovation Suggestion Logic
    renovation <- "🔹 Consider upgrading kitchen or adding a garage for higher ROI.\n"
    roi <- 0
    
    # If Kitchen is of lower quality
    if (input$kitchen_qual == "Fa") {
      renovation <- "✅ Upgrading the kitchen will significantly increase home value.\n"
      roi <- 0.10
    }
    if (input$kitchen_qual == "TA" | input$kitchen_qual == "Gd" ) {
      renovation <- "✅ Upgrading the interior of kitchen will be beneficial.\n"
      roi <- 0.10
    }
    
    # If Garage is missing
    if (input$garage_cars == 0) {
      renovation <- paste(renovation, "✅ Adding a garage will significantly increase home value.\n")
      roi <- roi + 0.07
    }
    
    # Living Area vs. Lot Area
    if (input$gr_liv_area < (0.6 * input$lot_area)) {
      renovation <- paste(renovation, "✅ Your home has a large lot with small living area. Consider adding a garden, fireplace, or pool for better ROI.\n")
      roi <- roi + 0.05
    }
    
    # Overall Quality-Based Recommendations
    if (input$overall_qual < 3) {
      renovation <- paste(renovation, "✅ Your home's overall quality is low. Painting and wall proofing are recommended to boost property value.\n")
      roi <- roi + 0.08
    } else if (input$overall_qual < 7) {
      renovation <- paste(renovation, "✅ Interior decoration and home improvements are suggested for better resale value.\n")
      roi <- roi + 0.06
    }
    
    # ROI Estimate
    price_after_renovation <- predicted_price * (1 + roi)
    
    output$predicted_price <- renderText({ paste("💰 Predicted Price: Rs.", format(round(predicted_price, 2), big.mark = ",")) })
    output$renovation_suggestion <- renderText({ renovation })
    output$roi_estimate <- renderText({ paste("📈 Estimated Price After Renovation: Rs.", format(round(price_after_renovation, 2), big.mark = ",")) })
    
    # ROI Plot (Bar Chart)
    output$roi_plot <- renderPlotly({
      ggplot(renovation_costs, aes(x = Renovation, y = Impact, fill = Renovation)) +
        geom_bar(stat = "identity") +
        labs(title = "Cost vs Impact of Renovations",
             x = "Renovation Type",
             y = "Price Impact (%)") +
        scale_y_continuous(labels = scales::percent) +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1))
    })
  })
}

# Run the app
shinyApp(ui = ui, server = server)



Listening on http://127.0.0.1:4039

